# The Vikings Network

In [7]:
import numpy as np
import sciris as sc
import starsim as ss

ss_float_ = ss.dtypes.float
ss_int_ = ss.dtypes.int



### Class: `VikingsNet`
This class is designed to model a dynamic network where vikings are the agents, and their interactions are captured as connections. The network evolves over time, with pairs of vikings being connected for specific durations. It inherits from `ss.DynamicNetwork`.

#### `__init__(self, key_dict=None, preBoard=True, postBoard=False, **kwargs)`
- **Purpose:** This is the constructor method that initializes the `VikingsNet` object.
- **Parameters:**
  - `key_dict`: A dictionary that defines network edge attributes, such as the duration (`dur`), start time (`start`), and end time (`end`) for interactions between vikings.
  - `preBoard`: A flag that indicates if certain network setup tasks should be done before the simulation starts.
  - `postBoard`: A flag for tasks to perform after the simulation ends.
- **Functionality:** 
  - It calls `sc.mergedicts()` to merge any custom keys provided in `key_dict` with the default ones: `dur`, `start`, and `end`.
  - It initializes the network by calling the superclass constructor (this is, the DynamicsNetwork base class), passing in the `key_dict`, `preBoard`, and `postBoard` flags along with any additional arguments (`kwargs`).

#### `init_pre(self, sim)`
- **Purpose:** Prepares the network for the simulation.
- **Functionality:** 
  - Calls `super().init_pre(sim)`, which initializes the network structure before the simulation begins. This is important for setting up the network at time step 0.

#### `add_pairs(self, viking_inds=None, second_viking_inds=None, dur=None, start=None)`
- **Purpose:** Adds pairs of vikings to the network, representing connections between them.
- **Parameters:**
  - `viking_inds`: The first set of vikings involved in interactions.
  - `second_viking_inds`: The second set of vikings involved in interactions.
  - `dur`: The duration for which each pair remains connected.
  - `start`: The time step at which each interaction starts. If not provided, it defaults to the current simulation time (`self.sim.ti`).
- **Functionality:** 
  - If no vikings are provided (`viking_inds is None`), the method returns 0 (no connections are made).
  - For each pair of vikings, it sets `beta` to 1, indicating active transmission of interaction between them, and calculates the `end` time for the interaction as `start + 2`.
  - Finally, it appends the connection to the network with details about the participating vikings (`p1`, `p2`), the interaction strength (`beta`), duration (`dur`), and timing (`start`, `end`).
  
#### `update(self)`
- **Purpose:** Updates the network by modifying connection properties as the simulation progresses.
- **Functionality:** 
  - The method sets `beta` (the interaction strength) to 0 for any connections where the interaction duration has been completed (i.e., when `end` time is less than or equal to the current simulation time).
  - This simulates the end of interaction or disease transmission between vikings in the network.

#### `end_pairs(self)`
- **Purpose:** Cleans up network edges by removing inactive connections.
- **Functionality:** 
  - The method identifies "active" connections as those where:
    - The interaction hasn't ended yet (`self.edges.end > self.sim.ti`), and
    - Both vikings involved in the interaction are still alive (`people.alive[self.edges.p1] & people.alive[self.edges.p2]`).
  - It removes any inactive edges by updating the internal data structures that track edges.
  - It returns the number of active connections remaining.

### Overall Explanation:
The `VikingsNet` class represents a network of interactions between vikings, dynamically evolving throughout the simulation. Pairs of vikings are connected for a specified duration, during which interactions occur (e.g., representing the spread of disease). As the simulation progresses, connections are updated or removed based on their duration and the status of the individuals involved. This class is highly flexible, allowing dynamic pairings, variable durations, and time-based updates to the network.


In [8]:

class VikingsNet(ss.DynamicNetwork):
    """
    Vikings Network
    """
    def __init__(self, key_dict=None, preBoard=True, postBoard=False, **kwargs):
        """
        Initialized empty and filled with vikings arrival throughout the simulation
        """
        key_dict = sc.mergedicts(dict(dur=ss_float_, start=ss_int_, end=ss_int_), key_dict)
        super().__init__(key_dict=key_dict, preBoard=preBoard, postBoard=postBoard, **kwargs)
        return

    def init_pre(self, sim):
        " Initialize the network "
        super().init_pre(sim)
        return
    
    def add_pairs(self, viking_inds=None, second_viking_inds=None, dur=None, start=None):
        " Add connections between vikings and newly arrived vikings to pair them into ship's doorms"
        if viking_inds is None:
            return 0
        else:
            if start is None:
                start = np.full_like(dur, fill_value=self.sim.ti)
            n = len(viking_inds)
            beta = np.ones(n)
            end =  start + 2 
            self.append(p1=viking_inds, p2=second_viking_inds, beta=beta, dur=dur, start=start, end=end)
            return n
        
    def update(self):
        """
        Set beta to 0 for vikings who complete duration of disease transmission
        """
        inactive = self.edges.end <= self.sim.ti
        self.edges.beta[inactive] = 0
        return

    def end_pairs(self):
        people = self.sim.people
        active = (self.edges.end > self.sim.ti) & people.alive[self.edges.p1] & people.alive[self.edges.p2]
        for k in self.meta_keys():
            self.edges[k] = self.edges[k][active]
        return len(active)
        


# Defining the simulation
### Overall Summary:
This code block sets up and initializes a simulation with the following key elements:
- **Disease dynamics** using an `SIR` model with a 10% initial disease prevalence and a transmission rate (`beta`) of 0.01.
- **Demographic processes** with a population of 500 agents, where new births occur every 0.25 years.
- The simulation starts on January 1st, 2000, and runs for 2 years, updating every 2 days.
- Although the network model is not explicitly defined here (a random network is commented out), it implies the use of a custom network like `VikingsNet`. 

#### 1. `pars = dict(...)`
This block defines a dictionary named `pars`, which contains various parameters for the simulation.

- **`diseases`:**
  - This parameter specifies the type of disease model being used. In this case, it's an `SIR` model (Susceptible, Infected, Recovered).
  - `ss.SIR(unit='day', dt=1.0, init_prev=0.1, beta=ss.beta(0.01))`: 
    - **`unit='day'`**: The time unit for disease transmission is in days.
    - **`dt=1.0`**: The time step for disease progression is 1 day.
    - **`init_prev=0.1`**: The initial prevalence of the disease in the population is 10% (0.1).
    - **`beta=ss.beta(0.01)`**: This defines the disease transmission rate (or infectivity) as 0.01. The `ss.beta()` function is used to calculate this value, which indicates how likely a disease is transmitted between agents per contact.

- **`demographics`:**
  - This parameter defines the demographic processes happening in the simulation.
  - `ss.Births(unit='year', dt=0.25)`: 
    - **`unit='year'`**: The time unit for demographic events like births is in years.
    - **`dt=0.25`**: The time step for demographic events is 0.25 years (3 months). This parameter ensures the population can grow through births at this interval.

- **`n_agents = 500`:**
  - Agents to be simulated.

#### 2. `sim = ss.Sim(pars, unit='day', dt=2, start='2000-01-01', stop='2002-01-01')`
- **`sim = ss.Sim(...)`:**
  - This line initializes the simulation with the parameters defined in the `pars` dictionary.

- **Arguments:**
  - **`pars`**: The dictionary of parameters defined earlier, which includes the disease model, demographics, start time, and agent count.
  - **`unit='day'`**: This sets the time unit for the entire simulation as days. It means that each step in the simulation corresponds to one day.
  - **`dt=2`**: This sets the time step to 2 days, meaning the simulation updates every 2 days (rather than every single day).
  - **`start='2000-01-01'`**: This sets the exact start date for the simulation: 
  - **`stop='2002-01-01'`**: This sets the stop date for the simulation: The simulation will run for 2 years.

#### 3. `sim.init()`
- **Purpose:** 
  - This initializes the simulation before it runs. It prepares the simulation environment, such as setting up the agents, initial conditions (like disease prevalence), and demographic parameters.
  
- **Functionality:**
  - The `init()` function is responsible for loading and setting up the simulation according to the parameters provided. It ensures that everything is ready to run the simulation from the start date.



In [9]:
pars = dict(
    diseases = ss.SIR(unit='day', dt=1.0, init_prev=0.1, beta=ss.beta(0.01)),
    demographics = ss.Births(unit='year', dt=0.25),
    start = '2000',
    # networks = ss.RandomNet(unit='week'), # We are going to use our own network. 
    n_agents = 500,
)
sim = ss.Sim(pars, unit='day', dt=2, start='2000-01-01', stop='2002-01-01')
sim.init()


Initializing sim with 500 agents


Sim(n=500; 2000-01-01—2002-01-01; demographics=births; diseases=sir)

# Adding the network to the simulation

This code sets up and integrates the custom `VikingsNet` network into the simulation, adds specific pairs of vikings to the network, and prints out the network object for inspection.

#### 1. `my_network = VikingsNet()`
This initializes an instance of the `VikingsNet` class.

#### 2. `my_network.init_pre(sim)`
- This connects the `VikingsNet` network to the simulation object (`sim`).
- The `init_pre()` method is called to register the network with the simulation. This prepares the network to interact with the simulation as it runs. The `init_pre()` function ensures that the network is integrated into the simulation environment, and the network can now dynamically evolve with the simulation.

#### 3. `my_network.add_pairs(viking_inds=[1, 2, 3], second_viking_inds=[100, 101, 102], dur=[5, 1, 1])`
- This adds specific pairs of vikings to the network.
  - `viking_inds=[1, 2, 3]` are the indices of the first set of vikings.
  - `second_viking_inds=[100, 101, 102]` are the indices of the second set of vikings, each of which is paired with a viking from the first list.
  - `dur=[5, 1, 1]` represents the duration of the connections between each pair of vikings. In this case, the connection between viking 1 and 100 lasts for 5 time units, while the other pairs last for 1 time unit each.

This step establishes interactions between the specified pairs of vikings, which will last for the specified duration. These interactions are now part of the network, allowing events like disease transmission to occur between the paired vikings.

#### 4. `net = sc.objdict(my_network=my_network)`
This creates an object dictionary (`sc.objdict`) that stores the `my_network` instance. 
`sc.objdict()` is a utility function (Sciris, a package often used with simulations) that allows for flexible object management. It creates a dictionary-like object where `my_network` is a key, and the `VikingsNet` instance is the corresponding value. This can be used to manage and access network objects within the simulation.


This code integrates the `VikingsNet` network into the simulation, pairs specific vikings with each other for given durations, and stores the network in an object dictionary for easy access. 

In [10]:

my_network = VikingsNet()
my_network.init_pre(sim)    # Register the network with the simulation object ( this is calling module.init_pre() - network is a module - to connect the network to the sim)
my_network.add_pairs(viking_inds=[1, 2, 3], second_viking_inds=[100, 101, 102], dur=[5, 1, 1])
net = sc.objdict( my_network=my_network)

print(net)


#0. 'my_network':
vikingsnet("vikingsnet", p1, p2, beta, dur, start, end, preBoard, postBoard)
   p1   p2  beta  dur  start  end
0   1  100   1.0  5.0      0    2
1   2  101   1.0  1.0      0    2
2   3  102   1.0  1.0      0    2


# Removing pairs
Now we simulate the death of an agent and invoke the termination of the corresponding pair. 

In [11]:

sim.people.alive[100] = False  # Viking dies 
my_network.end_pairs()         # Remove connections from non active vikings.


3

# Adding new pairs

during the simulation, we can also add new pairs to our network:

In [12]:
# four more vikings arrive

my_network.add_pairs(viking_inds=[7, 8], second_viking_inds=[103, 104], dur=[2, 2])  
net2 = sc.objdict( my_network=my_network)

print(net2)


#0. 'my_network':
vikingsnet("vikingsnet", p1, p2, beta, dur, start, end, preBoard, postBoard)
   p1   p2  beta  dur  start  end
0   2  101   1.0  1.0      0    2
1   3  102   1.0  1.0      0    2
2   7  103   1.0  2.0      0    2
3   8  104   1.0  2.0      0    2


### Now you can try to use this network on implementing diseases like:


Common diseases among the Vikings, like other historical populations, were largely influenced by their environment, 
lifestyle, diet, and living conditions. Based on archaeological evidence, historical records, 
and modern studies of Viking remains, some of the common diseases and health issues that Vikings may have faced include:

2. Tuberculosis (TB)
Description: A bacterial infection that primarily affects the lungs but can also impact other parts of the body. 
            It spreads through the air and would have been more common in crowded living conditions.
Evidence:   DNA evidence from Viking-era skeletons shows traces of tuberculosis bacteria, indicating its presence in Viking populations.

3. Leprosy (Hansen’s Disease)
Description: A chronic bacterial infection that affects the skin, nerves, and respiratory tract. 
            Leprosy was common in medieval Europe, and there is evidence that it affected the Vikings.
Evidence:   Archaeological finds in Scandinavian settlements reveal skeletal deformities consistent with leprosy, 
            and DNA analysis has confirmed the presence of the disease.

4. Parasites and Worms
Description: Vikings, like other medieval populations, likely suffered from intestinal parasites 
            (e.g., tapeworms, roundworms) due to poor sanitation and the consumption of undercooked meat or contaminated water.
Evidence:   Excavations of Viking latrines and burial sites have uncovered eggs of parasitic worms, 
            indicating that intestinal parasites were common.

8. Infectious Diseases (e.g., Plague)
Description: Vikings likely suffered from a range of infectious diseases, including viral and bacterial 
            infections such as smallpox and plague. Vikings engaged in extensive trade and raiding, which exposed them to diseases from different regions.
Evidence:   DNA from Viking skeletons in Norway has shown evidence of Yersinia pestis, 
            the bacterium responsible for the plague, indicating that the Vikings were exposed to the disease.

Viking Health Practices:
Herbal Medicine: Vikings relied on natural remedies using herbs and plants, many of which were based on local knowledge and passed down through generations.
Rituals and Amulets: Vikings also believed in the protective power of amulets, rituals, and Norse gods to ward off illness and disease, often mixing spiritual and practical approaches to health.
In summary, diseases common among Vikings were influenced by their harsh environment, physical lifestyle, and the close-knit, communal living conditions they often experienced. Many health issues they faced are consistent with what we would expect from a medieval, seafaring population.

